In [ ]:
# 利用するライブラリ
import matplotlib.pyplot as plt
import numpy as np
import copy
import time
import os
from tqdm import tqdm
from PIL import Image

import torch.nn as nn
from torch.autograd import Variable
import torch

import torchvision.transforms as transforms
import torchvision.models as models
import torchvision

In [ ]:
transform = transforms.Compose(
            [
            # 画像サイズが異る場合は利用
            # transforms.Resize((256,256)),
             transforms.RandomHorizontalFlip(),
             transforms.ToTensor()   
             ])

In [ ]:
# google colab等で実行する際にフォルダ内に .ipynb_checkpoints があると、ラベルの対象になるので削除
!rm -rf golf/.ipynb_checkpoints

In [ ]:
data = torchvision.datasets.ImageFolder(root='./golf', transform=transform)

In [ ]:
data.class_to_idx

In [ ]:
train_size = int(0.8 * len(data))

In [ ]:
val_size  = len(data) - train_size

In [ ]:
data_size  = {"train":train_size, "val":val_size}

In [ ]:
data_train, data_val = torch.utils.data.random_split(data, [train_size, val_size])

In [ ]:
train_loader = torch.utils.data.DataLoader(data_train, batch_size=8, shuffle=True)
val_loader   = torch.utils.data.DataLoader(data_val,   batch_size=8, shuffle=False)
dataloaders  = {"train":train_loader, "val":val_loader}

In [ ]:
def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# 訓練データをランダムに取得
dataiter = iter(dataloaders["train"])
images, labels = dataiter.next()

print(labels)

# 画像の表示
imshow(torchvision.utils.make_grid(images))
# ラベルの表示
print(' '.join('%5s' % labels[j].numpy() for j in range(8)))

In [ ]:
model = models.resnet18(pretrained=True)

In [ ]:
model

In [ ]:
for p in model.parameters():
    p.requires_grad=False

model.fc = nn.Linear(512, 2)

In [ ]:
# CPU環境の場合は不要
model = model.cuda()
lr = 1e-4
epoch = 40
optim = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)

# CPU環境の場合は cuda() は不要
criterion = nn.CrossEntropyLoss().cuda()

In [ ]:
def train_model(model, criterion, optimizer, scheduler=None, num_epochs=25):
    since = time.time() 

    for epoch in tqdm(range(num_epochs)):
        epoch_loss = 0
        epoch_acc = 0

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for data in dataloaders[phase]:
                inputs, labels = data

                # CPU環境では不要
                inputs = inputs.cuda()
                labels = labels.cuda()

                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()


                running_loss += loss.item() * inputs.size(0) 
                running_corrects += torch.sum(preds == labels)

            epoch_loss = running_loss / data_size[phase]
            epoch_acc = running_corrects.item() / data_size[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

    return model

In [ ]:
trained_model = train_model(model, criterion, optim, num_epochs=epoch)

In [ ]:
# 保存時
torch.save(trained_model.state_dict(), './golf-model.pth')

In [ ]:
# 学習済みロード
# model.load_state_dict(torch.load('./golf-model.pth'))

In [ ]:
# 予測
trained_model.eval()
imsize = 256
loader = transforms.Compose([transforms.Scale(imsize), transforms.ToTensor()])

def image_loader(image_name):
    """load image, returns cuda tensor"""
    image = Image.open(image_name).convert("RGB")
    image = loader(image)
    image = Variable(image, requires_grad=True)
    image = image.unsqueeze(0)  
    return image.cuda()

m = nn.Softmax(dim=1)

image = image_loader('./positive.png')
print(m(model(image)))

image = image_loader('./negative.png')
print(m(model(image)))